In [1]:
import json
import os
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from itertools import combinations
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

In [2]:
# # Comman function for init driver
# def init_driver(url):
#     driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH)
#     driver.maximize_window()
#     driver.get('https://paytm.com/flights')
#     driver.get(url)
#     return driver

## https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    try:
        driver.get(url)
    except:
        print('error 4')
            
    return driver


def wait_for_page_load(xpath_url):
    global driver
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
        return True
    except TimeoutException:
        print("Timed out waiting for page to load")
        try:
            driver.close()
        except:
            driver = init_driver(url)
            print('driver not found')
            
        # sleep(2)
        
        return False
    except NoSuchElementException:
        print("Element not found")
        return False


In [3]:
print('Starting...')
url = 'https://paytm.com/flights'
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
# chrome_options.headless = True
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
# chrome_options.add_argument('--user-data-dir=./User_Data')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--remote-debugging-port=9222")
# chrome_options.add_experimental_option("excludeSwitches",["ignore-certificate-errors"])

print('initiated...')

preferences = {
	# "profile.default_content_settings.popups": 0,
	"download.default_directory": os.getcwd() + os.path.sep,
	# "directory_upgrade": True
}

chrome_options.add_experimental_option('prefs', preferences)

# chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
driver.maximize_window()


driver.get(url)



Starting...
initiated...


In [4]:
#1500

In [5]:
pd.options.display.max_rows = 100

In [6]:
# all city list
citys = [
    'DEL-Delhi',
    'BOM-Mumbai',
    'BLR-Bengaluru',
    'HYD-Hyderabad',
    'CU-Kolkata',
    'MAA-Chennai',
    'AMD-Ahmedabad',
    'GOI-Goa',
    'PAT-Patna',
    'COK-Kochi',
    'LKO-Lucknow',
    'GAU-Guwahati',
    'PNQ-Pune',
    'JAI-Jaipur',
    'SXR-Srinagar',
    'BBI-Bhubaneshwar',
    'IXB-Bagdogra',
    'VNS-Varanasi',
    'IXC-Chandigarh',
    'IXR-Ranchi',
    'VTZ-Vishakhapatnam',
    'RPR-Raipur',
    'NAG-Nagpur',
    'TRV-Thiruvananthapuram',
    'CCJ-Kozhikode',
    'IDR-Indore',
    'ATQ-Amritsar',
    'IXJ-Jammu',
    'CJB-Coimbatore',
    'DED-Dehradun',
]

In [7]:
possible_combinations = list(combinations(citys,2))

In [8]:
def collect_data(driver, today, next_date, day_before):
    
    wait_for_page_load('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    wait_for_page_load('//div[@class="_3H-S _1Eia"]')
    
    all_boxes = driver.find_elements_by_xpath('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    try:
        for one_box in all_boxes:
            try:
                flight_name = one_box.find_element_by_xpath('.//div[@class="_3H-S _1Eia"]').text
                flight_id = one_box.find_element_by_xpath('.//div[@class="NqXj _2GoO"]').text
                start_time = one_box.find_element_by_xpath('.//div[@class="_3H-S"]').text
                end_time = one_box.find_element_by_xpath('.//div[@class="_3H-S _1wD5"]').text.split('\n')[0]
                travel_time = one_box.find_element_by_xpath('.//div[@class="vY4t"]').text
                stops = one_box.find_element_by_xpath('.//div[@class="_7BOG"]').text
                fare = one_box.find_element_by_xpath('.//div[@class="_1cxG"]').text

                all_data.append({
                    'flight_name': flight_name,
                    'flight_id': flight_id,
                    'start_time': start_time,
                    'end_time': end_time,
                    'travel_time': travel_time,
                    'stops': stops,
                    'fare': fare,
#                     'today': today,
#                     'next_date': next_date,
#                     'day_before': day_before
                })
            except Exception as e:
                print(e)
                print('error2')
                # sleep(5)
            
    except Exception as e:
        print(e)
        print('error3')
        # sleep(5)

In [9]:
def search_page(driver, source, destination):
    
    try:
        clear_inputs = driver.find_elements_by_xpath('//div[@class="fl-input-clear"]')
        clear_inputs[0].click()
        clear_inputs[1].click()
    except:
        print('cleared already')
        
    sleep(2)
    wait_for_page_load('//div[@class="_31PN"]//input')
    
    inputs = driver.find_elements_by_xpath('//div[@class="_31PN"]//input')

    start_input = inputs[0]
    end_input = inputs[1]

    start_input.clear()
    start_input.send_keys(source)
    sleep(2)
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    end_input.clear()
    end_input.send_keys(destination)
    sleep(2)
    
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    driver.find_element_by_xpath("//button[text()='Search']").click()
    sleep(2)
    

In [10]:
all_data = []
for i, one_combination in enumerate(possible_combinations):
    source, destination = one_combination
    today = datetime.today()
#     for i in range(1, 11):
    try:
        next_date = today.strftime('%Y-%m-%d') # (today + timedelta(i)).strftime('%Y-%m-%d')
        # url = f'https://paytm.com/flights/flightSearch/{source}/{destination}/1/0/0/E/{next_date}'
        # driver.get(url)
        search_page(driver, source, destination)
        sleep(2)
        collect_data(driver, today, next_date, 0)
    except Exception as e:
        # sleep(2)
        print(e)
        print('error1')
        try:
            driver.close()
        except:
            print('driver not found')
            
        driver = init_driver(url)
        
    print(one_combination, i+1, len(possible_combinations), (i+1)/ len(possible_combinations))
df = pd.DataFrame(all_data)
df

cleared already
('DEL-Delhi', 'BOM-Mumbai') 1 435 0.0022988505747126436
('DEL-Delhi', 'BLR-Bengaluru') 2 435 0.004597701149425287
('DEL-Delhi', 'HYD-Hyderabad') 3 435 0.006896551724137931
('DEL-Delhi', 'CU-Kolkata') 4 435 0.009195402298850575
('DEL-Delhi', 'MAA-Chennai') 5 435 0.011494252873563218
('DEL-Delhi', 'AMD-Ahmedabad') 6 435 0.013793103448275862
('DEL-Delhi', 'GOI-Goa') 7 435 0.016091954022988506
('DEL-Delhi', 'PAT-Patna') 8 435 0.01839080459770115
('DEL-Delhi', 'COK-Kochi') 9 435 0.020689655172413793
('DEL-Delhi', 'LKO-Lucknow') 10 435 0.022988505747126436
('DEL-Delhi', 'GAU-Guwahati') 11 435 0.02528735632183908
('DEL-Delhi', 'PNQ-Pune') 12 435 0.027586206896551724
('DEL-Delhi', 'JAI-Jaipur') 13 435 0.029885057471264367
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('DEL-Delhi', 'SXR-Srinagar') 14 435 0.03218390804597701
cleared already
('DEL-Delhi', 'BBI-Bhubaneshwar') 15 435 0.034482758620689655
('DEL-Delhi', 'IXB-Bagdogra') 16 435 

('CU-Kolkata', 'BBI-Bhubaneshwar') 121 435 0.27816091954022987
('CU-Kolkata', 'IXB-Bagdogra') 122 435 0.28045977011494255
('CU-Kolkata', 'VNS-Varanasi') 123 435 0.2827586206896552
('CU-Kolkata', 'IXC-Chandigarh') 124 435 0.2850574712643678
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('CU-Kolkata', 'IXR-Ranchi') 125 435 0.28735632183908044
cleared already
('CU-Kolkata', 'VTZ-Vishakhapatnam') 126 435 0.2896551724137931
('CU-Kolkata', 'RPR-Raipur') 127 435 0.29195402298850576
('CU-Kolkata', 'NAG-Nagpur') 128 435 0.2942528735632184
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('CU-Kolkata', 'TRV-Thiruvananthapuram') 129 435 0.296551724137931
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('CU-Kolkata', 'CCJ-Kozhikode') 130 435 0.2988505747126437
cleared already
('CU-Kolkata', 'IDR-Indore') 131 435 0.30114942528735633
('CU-Kolkata', 'ATQ-Amritsar') 

('PAT-Patna', 'TRV-Thiruvananthapuram') 219 435 0.503448275862069
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'CCJ-Kozhikode') 220 435 0.5057471264367817
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'IDR-Indore') 221 435 0.5080459770114942
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'ATQ-Amritsar') 222 435 0.5103448275862069
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'IXJ-Jammu') 223 435 0.5126436781609195
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'CJB-Coimbatore') 224 435 0.5149425287356322
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('PAT-Patna', 'DED-Dehradun

Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'CCJ-Kozhikode') 277 435 0.6367816091954023
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'IDR-Indore') 278 435 0.639080459770115
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'ATQ-Amritsar') 279 435 0.6413793103448275
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'IXJ-Jammu') 280 435 0.6436781609195402
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'CJB-Coimbatore') 281 435 0.6459770114942529
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('GAU-Guwahati', 'DED-Dehradun') 282 435 0.6482758620689655
cleared already
Timed out waiting f

Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'IXB-Bagdogra') 331 435 0.7609195402298851
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'VNS-Varanasi') 332 435 0.7632183908045977
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'IXC-Chandigarh') 333 435 0.7655172413793103
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'IXR-Ranchi') 334 435 0.767816091954023
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'VTZ-Vishakhapatnam') 335 435 0.7701149425287356
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('BBI-Bhubaneshwar', 'RPR-Raipur') 336 435 0.7724137931034483
cleared already
Timed out waiting for pa

('IXC-Chandigarh', 'DED-Dehradun') 380 435 0.8735632183908046
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranchi', 'VTZ-Vishakhapatnam') 381 435 0.8758620689655172
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranchi', 'RPR-Raipur') 382 435 0.8781609195402299
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranchi', 'NAG-Nagpur') 383 435 0.8804597701149425
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranchi', 'TRV-Thiruvananthapuram') 384 435 0.8827586206896552
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranchi', 'CCJ-Kozhikode') 385 435 0.8850574712643678
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXR-Ranch

Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('ATQ-Amritsar', 'IXJ-Jammu') 430 435 0.9885057471264368
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('ATQ-Amritsar', 'CJB-Coimbatore') 431 435 0.9908045977011494
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('ATQ-Amritsar', 'DED-Dehradun') 432 435 0.993103448275862
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXJ-Jammu', 'CJB-Coimbatore') 433 435 0.9954022988505747
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('IXJ-Jammu', 'DED-Dehradun') 434 435 0.9977011494252873
cleared already
Timed out waiting for page to load
Message: invalid session id

error1
driver not found
('CJB-Coimbatore', 'DED-Dehradun') 435 435 1.0


flight_name  flight_id start_time end_time travel_time  \
0       Go First   G8 - 354      22:30    00:40      2h 10m   
1       Go First   G8 - 323      18:45    21:05      2h 20m   
2       Go First   G8 - 717      19:45    08:25     12h 40m   
3        Vistara   UK - 981      21:40    23:50      2h 10m   
4         IndiGo  6E - 2137      23:35    01:45      2h 10m   
...          ...        ...        ...      ...         ...   
1885      IndiGo   6E - 547      22:30    07:05      8h 35m   
1886      IndiGo   6E - 122      23:35    08:15      8h 40m   
1887      IndiGo   6E - 122      23:35    08:55      9h 20m   
1888    Go First  G8 - 2608      21:15    07:30     10h 15m   
1889    Go First  G8 - 2608      21:15    11:25     14h 10m   

                    stops    fare  
0                Non Stop   5,954  
1                Non Stop   5,954  
2     1 stop at Ahmedabad   5,954  
3                Non Stop   5,955  
4                Non Stop   5,955  
...                   ...     ...  
1885  1 stop at Ahmedabad   7,260  
1886  1 stop at Hyderabad   8,006  
1887  1 stop at Hyderabad   8,321  
1888     1 stop at Mumbai  10,855  
1889     1 stop at Mumbai  10,855  

[1890 rows x 7 columns]

In [12]:
df.to_csv('flight_data_.csv')